# 체크용

In [2]:
import pandas as pd
import os

df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
print(len(df_match))

fs=os.listdir('../../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/')
len(fs)

10156


8804

In [15]:
df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_match = df_match.rename(columns={'Unnamed: 0':'index'})
#df_match.reset_index(inplace=True, drop=True)

print(len(df_match))
'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

nonlist=[]
yeslist=[]
srate= 100
for i in range(len(df_match)):
    filename = str(df_match.loc[i,'Value'])+','+str(df_match.loc[i,'index'])+','+ df_match.loc[i,'path']
    if not os.path.exists('../../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/'+filename):
        nonlist.append(df_match.loc[i,'path'])
    else:
        yeslist.append(df_match.loc[i,'path'])
        
len(nonlist),len(yeslist)

10156


(1352, 8804)

In [16]:
import pickle

cid = pickle.load(open('../../DL_model/caseids_diff','rb'))
cid = [e[:-3]+'vital' for e in cid]

existed = []
still=[]
for p in nonlist:
    if p in cid:
        existed.append(p)
    if p in yeslist:
        still.append(p)
len(cid),len(existed),len(still)

(591, 190, 1132)

# ECG 250Hz

In [ ]:
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np
import vitaldb

df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_match = df_match.rename(columns={'Unnamed: 0':'index'})
#df_match.reset_index(inplace=True, drop=True)

print(len(df_match))
'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

srate= 250
cnt=0
for i in range(len(df_match)):
    filename = str(df_match.loc[i,'Value'])+','+str(df_match.loc[i,'index'])+','+ df_match.loc[i,'path']
    
    if not os.path.exists('../../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/'+filename):
        cnt +=1
        print(cnt,' : ', i, end=' ')
        #try: 
        rec_path = '../../../cranberry2/Preprocessing/vital_data/pacu_ecg_pleth/pacu_ecg_pleth/'+df_match.loc[i,'path']
        #print(rec_path)
        #print('1')
        vfile_ecg = vitaldb.vital_recs_time(rec_path,['SNUADCW/ECG_II'],interval=0.004).flatten()

        #print(vfile)
        if len(vfile_ecg[0])==0 :
            print('index ',df_match.loc[i,'index'],' OPID ',df_match.loc[i,'opid'],'empty vital file')
        else:
            Abs_time = [vfile_ecg[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=i/srate) for i in range(len(vfile_ecg[0]))]
            #print(vfile[1])
            dics = {'Abs_time':Abs_time,'ECG':vfile_ecg[0]}
            df_vital = pd.DataFrame(data=dics)
            end_idx_list = df_vital[(df_vital['Abs_time'] > df_match.loc[i,'NRS_time'] - datetime.timedelta(seconds=1)) & (df_vital['Abs_time'] < df_match.loc[i,'NRS_time'] + datetime.timedelta(seconds=1))].index.tolist()

            if len(end_idx_list)==0:
                print('index ',i,' OPID ',df_match.loc[i,'opid']," no vital data at NRS time")

            else:
                end_idx=end_idx_list[0]+1
                start_idx= end_idx - 5 * 60 * srate

                ext = df_vital[start_idx:end_idx]
                extr=ext.dropna(subset=['ECG'])
                extr=extr.drop_duplicates(subset=['ECG'])

                if len(extr)>1:
                    if start_idx <0 :
                        fit = (-1)*end_idx 
                        extraction = pd.DataFrame(index=range(5*60*srate),columns=['Abs_time','ECG'])
                        extraction[fit:]=ext

                        print(extr)
                        print(df_match.loc[i,'path'])

                    else:
                        extraction = ext
                    #with open('vital_data/ECG_250Hz_pacu_5min/'+filename,'wb') as f:
                        #pickle.dump(extraction, f)
                    print('...save error')
                else:
                    print('... 결측치만 있거나 직선')
        #except:
            #print('error: ',i,df_match.loc[i,'opid'],df_match.loc[i,'path'])

10156
1  :  9 ... 결측치만 있거나 직선
2  :  23 ... 결측치만 있거나 직선
3  :  27 ... 결측치만 있거나 직선
4  :  37 ... 결측치만 있거나 직선
5  :  45 ... 결측치만 있거나 직선
6  :  47 ... 결측치만 있거나 직선
7  :  60 ... 결측치만 있거나 직선
8  :  70 ... 결측치만 있거나 직선
9  :  75 ... 결측치만 있거나 직선
10  :  81 ... 결측치만 있거나 직선
11  :  83 ... 결측치만 있거나 직선
12  :  84 ... 결측치만 있거나 직선
13  :  85 ... 결측치만 있거나 직선
14  :  86 ... 결측치만 있거나 직선
15  :  87 ... 결측치만 있거나 직선
16  :  94 ... 결측치만 있거나 직선
17  :  98 ... 결측치만 있거나 직선
18  :  101 ... 결측치만 있거나 직선
19  :  103 ... 결측치만 있거나 직선
20  :  111 ... 결측치만 있거나 직선
21  :  112 ... 결측치만 있거나 직선
22  :  113 ... 결측치만 있거나 직선
23  :  114 ... 결측치만 있거나 직선
24  :  116 ... 결측치만 있거나 직선
25  :  117 ...save error
26  :  120 ... 결측치만 있거나 직선
27  :  144 ... 결측치만 있거나 직선
28  :  154 ... 결측치만 있거나 직선
29  :  156 ... 결측치만 있거나 직선
30  :  157 ... 결측치만 있거나 직선
31  :  160 ... 결측치만 있거나 직선
32  :  166 ... 결측치만 있거나 직선
33  :  169 ... 결측치만 있거나 직선
34  :  174 ... 결측치만 있거나 직선
35  :  175 ... 결측치만 있거나 직선
36  :  176 ... 결측치만 있거나 직선
37  :  177 ... 결측치만 있거나 직선
38  :  186 ... 결측치만 있거나 